In [ ]:
!pip -q install geemap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.5 MB/s eta 0:00:00


In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='slum-mapping')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import json
gpath = "/content/drive/My Drive/Colab Notebooks/nairobi_extent1km.geojson"  # adjust if needed
with open(gpath, "r") as f:
    gj = json.load(f)

fc  = ee.FeatureCollection(gj["features"])
AOI = fc.geometry()

Mounted at /content/drive


In [ ]:
START_DATE = "2025-03-03"
END_DATE   = "2025-03-04"   # end exclusive → includes Mar 3

s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(AOI)
      .filterDate(START_DATE, END_DATE)
      .sort("CLOUDY_PIXEL_PERCENTAGE"))  # clearest first
img = ee.Image(s2.first()).clip(AOI)

In [ ]:
import geemap

vis_true = {"bands": ["B4","B3","B2"], "min": 0, "max": 3000}   # true color
vis_swir = {"bands": ["B12","B8","B4"], "min": 0, "max": 3000}  # SWIR/NIR combo

# Center on AOI
center_latlon = AOI.centroid(10).coordinates().reverse().getInfo()
Map = geemap.Map(center=center_latlon, zoom=12)

Map.addLayer(img, vis_true, "S2 True Color (2025-03-03)")
Map.addLayer(img, vis_swir, "S2 SWIR (B12,B8,B4)")
Map.addLayer(fc.style(color='cyan', fillColor='00000000', width=2), {}, "AOI 1 km buffer")
Map.addLayerControl()

Map  # ← this displays the live map

Map(center=[-1.2601421996562385, 36.815578065449415], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
import ee, folium
ee.Initialize(project='slum-mapping')
AOI = fc.geometry()

In [ ]:
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(AOI)
      .filterDate("2025-03-03", "2025-03-04")
      .sort("CLOUDY_PIXEL_PERCENTAGE"))
img = ee.Image(s2.first()).clip(AOI)

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1):
    map_id = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name, overlay=True, control=True, show=show, opacity=opacity
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

vis_true = {"bands": ["B4","B3","B2"], "min": 0, "max": 3000}
vis_swir = {"bands": ["B12","B8","B4"], "min": 0, "max": 3000}

center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)
m.add_ee_layer(img, vis_true, "S2 True Color (2025-03-03)", True)
m.add_ee_layer(img, vis_swir, "S2 SWIR (B12,B8,B4)", False)
m.add_child(folium.GeoJson(gj, name="AOI 1 km buffer"))
m.add_child(folium.LayerControl())
display(m)

# Save to HTML if you like
m.save("Nairobi_S2_20250303.html")

In [ ]:
# ---- Cloud / shadow helpers ----
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST   = 3    # km
BUFFER         = 70   # m

def add_cloud_bands(img):
    # Prefer joined s2cloudless; fall back to MSK_CLDPRB if join is missing.
    has_join = img.propertyNames().contains('s2cloudless')
    cld_prb = ee.Image(ee.Algorithms.If(
        has_join,
        ee.Image(img.get('s2cloudless')).select('probability'),
        img.select('MSK_CLDPRB')  # 0–100 cloud probability in S2_SR_HARMONIZED
    )).rename('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands([cld_prb, is_cloud])

def add_shadow_bands(img):
    not_water = img.select('SCL').neq(6)
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).And(not_water).rename('dark_pixels')
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))
    cld_proj = (img.select('clouds')
                  .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
                  .reproject(crs=img.select(0).projection(), scale=100)
                  .select('distance').mask().rename('cloud_transform'))
    shadows = cld_proj.And(dark_pixels).rename('shadows')
    return img.addBands([dark_pixels, cld_proj, shadows])

def add_cld_shdw_mask(img):
    img = add_cloud_bands(img)
    img = add_shadow_bands(img)
    is_cld_shdw = img.select('clouds').add(img.select('shadows')).gt(0)
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
                   .reproject(crs=img.select(0).projection(), scale=20)
                   .rename('cloudmask'))
    return img.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    not_cld_shdw = img.select('cloudmask').Not()
    # keep reflectance bands only
    return img.select('B.*').updateMask(not_cld_shdw)

# ---- SWIR (B11,B12) → 10 m resampling ----
def add_resampled_swir_10m(img, method="bilinear"):
    # method can be: 'nearest' | 'bilinear' | 'cubic'
    p10 = img.select('B2').projection()  # 10 m grid
    swir10 = (img.select(['B11','B12'])
                .resample(method)
                .reproject(crs=p10))     # align to the exact 10 m grid
    return img.addBands(swir10, overwrite=True)



In [ ]:


# best_clean = (ee.ImageCollection([img])
#               .map(add_cld_shdw_mask)
#               .map(apply_cld_shdw_mask)
#               .map(add_resampled_swir_10m)  # bilinear is default
#               .mosaic()
#               .clip(AOI))

best_clean = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
              .filterBounds(AOI)
              .filterDate("2025-03-03", "2025-03-04")
              .sort("CLOUDY_PIXEL_PERCENTAGE")
              .map(add_cld_shdw_mask)
              .map(apply_cld_shdw_mask)
              .map(add_resampled_swir_10m)
              .mosaic()
              .clip(AOI))


In [ ]:

import folium
from IPython.display import display

def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1):
    map_id = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name, overlay=True, control=True, show=show, opacity=opacity
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# --- Visualization parameters ---
vis_true = {"bands": ["B4","B3","B2"], "min": 0, "max": 3000}
vis_swir = {"bands": ["B12","B8","B4"], "min": 0, "max": 3000}

# --- Build and show the map ---
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

m.add_ee_layer(best_clean, vis_true, "S2 True Color (masked + bilinear)", True)
m.add_ee_layer(best_clean, vis_swir, "S2 SWIR (masked + bilinear)", False)

try:
    m.add_child(folium.GeoJson(shapefile, name="AOI 1 km buffer"))
except:
    pass

m.add_child(folium.LayerControl())
display(m)

# Optional: save to HTML
m.save("Nairobi_S2_masked_bilinear_20250303.html")

